In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

# = = = = = = = = = = = = = = =

option = ChromeOptions()

# = = = = = = = = = = = = = = =

option.add_extension(Proxy.get_Proxy_Selenium('coroutine'))
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)

                browser.maximize_window()

                browser.get(url)
                
                # = = = = = = = = = = = = = = =
                
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')

                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@class="part-detail-title"]')))
                title = browser.find_element(by=By.XPATH, value='//h1[@class="part-detail-title"]').text.strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="part-detail-title-number"]')))
                part_number = browser.find_element(by=By.XPATH, value='//p[@class="part-detail-title-number"]').text.strip()
                
                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//p[@class="item-description"]')))
                description = browser.find_element(by=By.XPATH, value='//p[@class="item-description"]').get_attribute('outerHTML')

                html = etree.HTML(description)
                description = ' '.join(html.xpath('//p[@class="item-description"]/text()')).strip()
                
                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//img[@id="primaryImage"]')))
                src = browser.find_element(by=By.XPATH, value='//img[@id="primaryImage"]').get_attribute('src').strip()
                
                # = = = = = = = = = = = = = = =
                
                list_name = browser.find_elements(by=By.XPATH, value='//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-9 large-6"]')
                list_value = browser.find_elements(by=By.XPATH, value='//div[@class="attribute-container flex-container flex-wrap"]/div[@class="small-12 medium-15 large-18"]')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Title': title,
                                         'Description': description,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])

                for name, value in zip(list_name, list_value):
                    df_temp.loc[0, name.text.strip()[:-1]] = value.text.strip()
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])
            
            browser.quit()

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(8):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：747

https://www.summitracing.com/parts/fel-70820  <->  [ok] - 剩余数量：737 - 当前时间：15:07:21
https://www.summitracing.com/parts/fel-2817  <->  [ok] - 剩余数量：736 - 当前时间：15:07:21
https://www.summitracing.com/parts/fel-os30790r  <->  [ok] - 剩余数量：735 - 当前时间：15:07:21
https://www.summitracing.com/parts/fel-260-1974  <->  [ok] - 剩余数量：734 - 当前时间：15:07:22
https://www.summitracing.com/parts/ham-0171  <->  [ok] - 剩余数量：733 - 当前时间：15:07:23
https://www.summitracing.com/parts/fel-cs26248  <->  [ok] - 剩余数量：732 - 当前时间：15:07:24
https://www.summitracing.com/parts/ham-0173  <->  [ok] - 剩余数量：731 - 当前时间：15:07:34
https://www.summitracing.com/parts/ham-2524  <->  [ok] - 剩余数量：730 - 当前时间：15:07:37
https://www.summitracing.com/parts/fel-os30697r  <->  [ok] - 剩余数量：729 - 当前时间：15:07:41
https://www.summitracing.com/parts/ham-2525  <->  [ok] - 剩余数量：728 - 当前时间：15:07:46
https://www.summitracing.com/parts/hed-45303  <->  [ok] - 剩余数量：727 - 当前时间：15:07:51
https://www.summitracing.com/parts/fel-cs26223  <->  [ok] - 剩余数量：726 - 当

https://www.summitracing.com/parts/mml-oslsx401120  <->  [ok] - 剩余数量：640 - 当前时间：15:15:35
https://www.summitracing.com/parts/irp-egm-243  <->  [ok] - 剩余数量：639 - 当前时间：15:15:38
https://www.summitracing.com/parts/mor-20105  <->  [ok] - 剩余数量：638 - 当前时间：15:15:44
https://www.summitracing.com/parts/mor-20138  <->  [ok] - 剩余数量：637 - 当前时间：15:15:44
https://www.summitracing.com/parts/irp-egm-202  <->  [ok] - 剩余数量：636 - 当前时间：15:15:51
https://www.summitracing.com/parts/mor-20142  <->  [ok] - 剩余数量：635 - 当前时间：15:15:55
https://www.summitracing.com/parts/mor-20144  <->  [ok] - 剩余数量：634 - 当前时间：15:15:55
https://www.summitracing.com/parts/mml-oslsx401014  <->  [ok] - 剩余数量：633 - 当前时间：15:15:59
https://www.summitracing.com/parts/mor-20148  <->  [ok] - 剩余数量：632 - 当前时间：15:16:06
https://www.summitracing.com/parts/icb-551666  <->  [ok] - 剩余数量：631 - 当前时间：15:16:09
https://www.summitracing.com/parts/hok-71222017hkr  <->  [ok] - 剩余数量：630 - 当前时间：15:16:12
https://www.summitracing.com/parts/mor-20146  <->  [ok] - 剩余数量：6

https://www.summitracing.com/parts/aaf-all26241  <->  [ok] - 剩余数量：543 - 当前时间：15:23:22
https://www.summitracing.com/parts/opg-pp07404-pl  <->  [ok] - 剩余数量：542 - 当前时间：15:23:26
https://www.summitracing.com/parts/opg-pp07404-bk  <->  [ok] - 剩余数量：541 - 当前时间：15:23:33
https://www.summitracing.com/parts/ado-10-9289  <->  [ok] - 剩余数量：540 - 当前时间：15:23:33
https://www.summitracing.com/parts/ado-10-9290  <->  [ok] - 剩余数量：539 - 当前时间：15:23:35
https://www.summitracing.com/parts/ado-11099271  <->  [ok] - 剩余数量：538 - 当前时间：15:23:42
https://www.summitracing.com/parts/ado-11570082  <->  [ok] - 剩余数量：537 - 当前时间：15:23:43
https://www.summitracing.com/parts/ado-12612350  <->  [ok] - 剩余数量：536 - 当前时间：15:23:51
https://www.summitracing.com/parts/ado-10-9246  <->  [ok] - 剩余数量：535 - 当前时间：15:23:53
https://www.summitracing.com/parts/ado-24203590  <->  [ok] - 剩余数量：534 - 当前时间：15:24:02
https://www.summitracing.com/parts/ado-12640746  <->  [ok] - 剩余数量：533 - 当前时间：15:24:04
https://www.summitracing.com/parts/ado-11562588  <-> 

https://www.summitracing.com/parts/cmb-08-0146  <->  [ok] - 剩余数量：446 - 当前时间：15:33:57
https://www.summitracing.com/parts/cmb-08-0142  <->  [ok] - 剩余数量：445 - 当前时间：15:34:00
https://www.summitracing.com/parts/cmb-08-0171  <->  [ok] - 剩余数量：444 - 当前时间：15:34:02
https://www.summitracing.com/parts/opg-pp07405  <->  [ok] - 剩余数量：443 - 当前时间：15:34:06
https://www.summitracing.com/parts/cmb-08-0129  <->  [ok] - 剩余数量：442 - 当前时间：15:34:11
https://www.summitracing.com/parts/pff-15-5001  <->  [ok] - 剩余数量：441 - 当前时间：15:34:22
https://www.summitracing.com/parts/opg-pp07405-st  <->  [ok] - 剩余数量：440 - 当前时间：15:34:22
https://www.summitracing.com/parts/pgp-prp-3454  <->  [ok] - 剩余数量：439 - 当前时间：15:34:30
https://www.summitracing.com/parts/pro-66277  <->  [ok] - 剩余数量：438 - 当前时间：15:34:39
https://www.summitracing.com/parts/cmb-08-0159  <->  [ok] - 剩余数量：437 - 当前时间：15:34:43
https://www.summitracing.com/parts/opg-pp07405-pl  <->  [ok] - 剩余数量：436 - 当前时间：15:34:43
https://www.summitracing.com/parts/pro-67054  <->  [ok] - 剩余

https://www.summitracing.com/parts/smp-tx3a  <->  [ok] - 剩余数量：350 - 当前时间：15:40:20
https://www.summitracing.com/parts/snh-ls14-bec  <->  [ok] - 剩余数量：349 - 当前时间：15:40:27
https://www.summitracing.com/parts/snh-ls14-sec  <->  [ok] - 剩余数量：348 - 当前时间：15:40:31
https://www.summitracing.com/parts/sum-120116  <->  [ok] - 剩余数量：347 - 当前时间：15:40:36
https://www.summitracing.com/parts/sad-sa516  <->  [ok] - 剩余数量：346 - 当前时间：15:40:36
https://www.summitracing.com/parts/sum-121200b  <->  [ok] - 剩余数量：345 - 当前时间：15:40:44
https://www.summitracing.com/parts/sum-121200  <->  [ok] - 剩余数量：344 - 当前时间：15:40:45
https://www.summitracing.com/parts/sum-121201b  <->  [ok] - 剩余数量：343 - 当前时间：15:40:54
https://www.summitracing.com/parts/sum-122241  <->  [ok] - 剩余数量：342 - 当前时间：15:41:02
https://www.summitracing.com/parts/rnb-65385  <->  [ok] - 剩余数量：341 - 当前时间：15:41:16
https://www.summitracing.com/parts/sum-01-3454  <->  [ok] - 剩余数量：340 - 当前时间：15:41:17
https://www.summitracing.com/parts/sum-150153  <->  [ok] - 剩余数量：339 - 当前时

https://www.summitracing.com/parts/trd-42231  <->  [ok] - 剩余数量：253 - 当前时间：15:53:25
https://www.summitracing.com/parts/trd-42233  <->  [ok] - 剩余数量：252 - 当前时间：15:53:33
https://www.summitracing.com/parts/trd-42234  <->  [ok] - 剩余数量：251 - 当前时间：15:53:36
https://www.summitracing.com/parts/trd-42241  <->  [ok] - 剩余数量：250 - 当前时间：15:53:48
https://www.summitracing.com/parts/trd-42242  <->  [ok] - 剩余数量：249 - 当前时间：15:53:58
https://www.summitracing.com/parts/trd-42164  <->  [ok] - 剩余数量：248 - 当前时间：15:55:11
https://www.summitracing.com/parts/trd-42236  <->  [ok] - 剩余数量：247 - 当前时间：15:55:33
https://www.summitracing.com/parts/trd-42243  <->  [ok] - 剩余数量：246 - 当前时间：15:55:51
https://www.summitracing.com/parts/trd-42036  <->  [ok] - 剩余数量：245 - 当前时间：15:55:52
https://www.summitracing.com/parts/trd-42253  <->  [ok] - 剩余数量：244 - 当前时间：15:56:02
https://www.summitracing.com/parts/sum-g3612b-k  <->  [ok] - 剩余数量：243 - 当前时间：15:56:05
https://www.summitracing.com/parts/trd-42921  <->  [ok] - 剩余数量：242 - 当前时间：15:56:13
h